In [1]:
# import
import requests
import pymongo
from pymongo import MongoClient
import time
import json
import pprint
import pandas as pd
from pandas.io.json import json_normalize
from datetime import timezone, timedelta, datetime
import logging
import logging.handlers
logger = logging.getLogger("crumbs")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(levelname)s|%(filename)s:%(lineno)s] %(asctime)s > %(message)s')
file_max_bytes = 10 * 1024 * 1024
fileHandler = logging.handlers.RotatingFileHandler(filename='./CoinTrader_Binance.log', maxBytes=file_max_bytes, backupCount=10)
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)
logger.info("CoinTrader Re-started")

In [2]:
base_url = 'https://api.binance.com'
symbol = 'BTCUSDT'
apis = {'ping':{'url': '/api/v1/ping','param':0},
        'time':{'url':'/api/v1/time','param':0},
        'exchangeInfo':{'url':'/api/v1/exchangeInfo','param':0},
        'depth':{'url':'/api/v1/depth','param':{'symbol':symbol}},
        'trades':{'url':'/api/v1/trades','param':{'symbol':symbol}},
        'aggTrades':{'url':'/api/v1/aggTrades','param':{'symbol':symbol}},
        'klines':{'url':'/api/v1/klines','param':{'symbol':symbol,'interval':'1m'}},
        '24hr':{'url':'/api/v1/ticker/24hr','param':{'symbol':symbol}},
        'price':{'url':'/api/v1/ticker/price','param':{'symbol':symbol}},
        'bookTicker':{'url':'/api/v3/ticker/bookTicker','param':{'symbol':symbol}},
       }
url_db = 'mongodb://mongo-7a3e43c9-1.bf9bfd39.cont.dockerapp.io:32794'
symbols = ['BTCUSDT','ETHUSDT','NEOUSDT','BNBUSDT','LTCUSDT','BCCUSDT']
intervals = ['1m','3m','5m','15m','30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1M']

In [6]:
client = MongoClient(url_db)
# Previous Binance DB Delete
db = client.Binance
api_name = 'klines'
symbol = 'BTCUSDT'
interval = '1h'
# DB & collection define
col_name = api_name+'_'+symbol+'_'+interval
col = db[col_name]
klines = col.find()
count = klines.count()
for idx,k in enumerate(klines):
    try:
        if idx==0:
            previous = k['o_t']
            continue
        gap = k['o_t']-previous
        if gap/1000!=3600:
            print(gap/1000/3600,datetime.fromtimestamp(k['o_t']/1000),datetime.fromtimestamp(previous/1000))
        previous = k['o_t']
    except:
        print('error')
        continue
        print([datetime.fromtimestamp(k['o_t']/1000) for k in klines[count-5:]])
client.close()

-4009.0 2017-08-17 13:00:00 2018-01-31 14:00:00
8.0 2017-09-07 08:00:00 2017-09-07 00:00:00
2.0 2018-01-04 14:00:00 2018-01-04 12:00:00


In [ ]:
# load DB
url_ticker = "https://api.coinone.co.kr/ticker/" # limit 90 req/min
payload = {"currency": "all"}
url_db = 'mongodb://localhost:27017/'

client = MongoClient(url_db)
db = client.CoinTracker
col = db.Ticker
col.count()
cursor = col.find()
df =  pd.DataFrame(json_normalize(list(cursor)))

In [ ]:
# pre-process

In [ ]:
len(df)

In [ ]:
for key in df.keys():print(key,df[key][0])

In [ ]:
# day = datetime.datetime.fromtimestamp(int(df.timestamp[0])).day
# year = datetime.datetime.fromtimestamp(int(df.timestamp[0])).year
# month = datetime.datetime.fromtimestamp(int(df.timestamp[0])).month
# weekday = datetime.datetime.fromtimestamp(int(df.timestamp[0])).weekday()
# hour = datetime.datetime.fromtimestamp(int(df.timestamp[0])).strftime('%I')
# minute = datetime.datetime.fromtimestamp(int(df.timestamp[0])).minute
# second = datetime.datetime.fromtimestamp(int(df.timestamp[0])).second
# ampm = datetime.datetime.fromtimestamp(int(df.timestamp[0])).strftime('%p')

In [ ]:
df["day"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).day for i in range(len(df))]
df["year"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).year for i in range(len(df))]
df["month"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).month for i in range(len(df))]
df["weekday"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).weekday() for i in range(len(df))]
df["hour"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).strftime('%I') for i in range(len(df))]
df["minute"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).minute for i in range(len(df))]
df["second"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).second for i in range(len(df))]
df["ampm"] = [datetime.datetime.fromtimestamp(int(df.timestamp[i])).strftime('%p') for i in range(len(df))]

In [ ]:
# time
timestamp = time.time()*1000
time_kor = datetime.fromtimestamp(timestamp/1000)
# year = now.year
# month = now.month
# day = now.day
# weekday = now.weekday()
# hour = now.strftime('%I')
# minute = now.minute
# second = now.second
# ampm = now.strftime('%p')

In [ ]:
# time error
api_name = 'time'
param = 0
url_ticker = base_url+apis[api_name]['url'] # 
r = requests.get(url_ticker,params=param)
timestamp = time.time()*1000
server_time = r.json()["serverTime"]
time_gap = (timestamp-server_time)/1000
# if time_gap > 1: continue
print(time_gap)

In [ ]:
term = 24*12

In [ ]:
df["mean.24"] = [df["btc.last"][i:i+term].mean() for i in range(len(df))]

In [ ]:
client.close()    
